In [1]:
import unicodedata
import pandas as pd
import string
import re
import numpy as np
import nltk
from nltk.util import ngrams
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import  SnowballStemmer
from nltk.stem import WordNetLemmatizer
from matplotlib import pyplot as plt

nltk.download('wordnet')
nltk.download('stopwords')

np.set_printoptions(linewidth=100)

%autosave 120

[nltk_data] Downloading package wordnet to
[nltk_data]     /home/yamasjose/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/yamasjose/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Autosaving every 120 seconds


In [2]:
twitter = pd.read_csv('data/gender-classifier-DFE-791531.csv', encoding='iso-8859-1' )

The dataset contains the following fields:

**unit_id:** a unique id for user

**golden:** whether the user was included in the gold standard for the model; TRUE or FALSE

**unit_state:** state of the observation; one of finalized (for contributor-judged) or golden (for gold standard observations)

**trusted_judgments:** number of trusted judgments (int); always 3 for non-golden, and what may be a unique id for gold standard observations

**last_judgment_at:** date and time of last contributor judgment; blank for gold standard observations

**gender:** one of male, female, or brand (for non-human profiles)

**gender:confidence:** a float representing confidence in the provided gender

**profile_yn:** "no" here seems to mean that the profile was meant to be part of the dataset but was not available when contributors went to judge it

**profile_yn:confidence:** confidence in the existence/non-existence of the profile

**created:** date and time when the profile was created

**description:** the user's profile description

**fav_number:** number of tweets the user has favorited

**gender_gold:** if the profile is golden, what is the gender?

**link_color:** the link color on the profile, as a hex value

**name:** the user's name

**profile_yn_gold:** whether the profile y/n value is golden

**profileimage:** a link to the profile image

**retweet_count:** number of times the user has retweeted (or possibly, been retweeted)

**sidebar_color:** color of the profile sidebar, as a hex value

**text:** text of a random one of the user's tweets

**tweet_coord:** if the user has location turned on, the coordinates as a string with the format "[latitude, longitude]"

**tweet_count:** number of tweets that the user has posted

**tweet_created:** when the random tweet (in the text column) was created

**tweet_id:** the tweet id of the random tweet

**tweet_location:** location of the tweet; seems to not be particularly normalized

**user_timezone:** the timezone of the user

In [3]:
pd.set_option('display.max_columns', None)

twitter.head(30)

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,gender,gender:confidence,profile_yn,profile_yn:confidence,created,description,fav_number,gender_gold,link_color,name,profile_yn_gold,profileimage,retweet_count,sidebar_color,text,tweet_coord,tweet_count,tweet_created,tweet_id,tweet_location,user_timezone
0,815719226,False,finalized,3,10/26/15 23:24,male,1.0000,yes,1.0,12/5/13 1:48,i sing my own rhythm.,0,NaN,08C2C2,sheezy0,NaN,https://pbs.twimg.com/profile_images/414342229...,0,FFFFFF,Robbie E Responds To Critics After Win Against...,NaN,110964,10/26/15 12:40,6.587300e+17,main; @Kan1shk3,Chennai
1,815719227,False,finalized,3,10/26/15 23:30,male,1.0000,yes,1.0,10/1/12 13:51,I'm the author of novels filled with family dr...,68,NaN,0084B4,DavdBurnett,NaN,https://pbs.twimg.com/profile_images/539604221...,0,C0DEED,ÛÏIt felt like they were my friends and I was...,NaN,7471,10/26/15 12:40,6.587300e+17,NaN,Eastern Time (US & Canada)
2,815719228,False,finalized,3,10/26/15 23:33,male,0.6625,yes,1.0,11/28/14 11:30,louis whining and squealing and all,7696,NaN,ABB8C2,lwtprettylaugh,NaN,https://pbs.twimg.com/profile_images/657330418...,1,C0DEED,i absolutely adore when louis starts the songs...,NaN,5617,10/26/15 12:40,6.587300e+17,clcncl,Belgrade
3,815719229,False,finalized,3,10/26/15 23:10,male,1.0000,yes,1.0,6/11/09 22:39,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",202,NaN,0084B4,douggarland,NaN,https://pbs.twimg.com/profile_images/259703936...,0,C0DEED,Hi @JordanSpieth - Looking at the url - do you...,NaN,1693,10/26/15 12:40,6.587300e+17,"Palo Alto, CA",Pacific Time (US & Canada)
4,815719230,False,finalized,3,10/27/15 1:15,female,1.0000,yes,1.0,4/16/14 13:23,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,37318,NaN,3B94D9,WilfordGemma,NaN,https://pbs.twimg.com/profile_images/564094871...,0,0,Watching Neighbours on Sky+ catching up with t...,NaN,31462,10/26/15 12:40,6.587300e+17,NaN,NaN
5,815719231,False,finalized,3,10/27/15 1:47,female,1.0000,yes,1.0,3/11/10 18:14,you don't know me.,3901,NaN,F5ABB5,monroevicious,NaN,https://pbs.twimg.com/profile_images/656336865...,0,0,"Ive seen people on the train with lamps, chair...",NaN,20036,10/26/15 12:40,6.587300e+17,New York Gritty,Central Time (US & Canada)
6,815719232,False,finalized,3,10/27/15 1:57,brand,1.0000,yes,1.0,4/24/08 13:03,"A global marketplace for images, videos and mu...",4122,NaN,298AAE,Shutterstock,NaN,https://pbs.twimg.com/profile_images/528547133...,0,0,@BpackEngineer Thank you for your patience whi...,NaN,13354,10/26/15 12:40,6.587300e+17,Worldwide,Eastern Time (US & Canada)
7,815719233,False,finalized,3,10/26/15 23:48,male,1.0000,yes,1.0,12/3/12 21:54,The secret of getting ahead is getting started.,80,NaN,0000FF,RobinMeske,NaN,https://pbs.twimg.com/profile_images/508875440...,0,C0DEED,Gala Bingo clubs bought for å£241m: The UK's l...,NaN,112117,10/26/15 12:40,6.587300e+17,NaN,NaN
8,815719234,False,finalized,3,10/27/15 1:52,female,1.0000,yes,1.0,9/8/15 4:50,Pll Fan // Crazy about MCD // Ramen is bae,1825,NaN,9266CC,pigzilla_,NaN,https://pbs.twimg.com/profile_images/658670112...,0,0,@_Aphmau_ the pic defines all mcd fangirls/fan...,NaN,482,10/26/15 12:40,6.587300e+17,NaN,NaN
9,815719235,False,finalized,3,10/27/15 1:49,female,1.0000,yes,1.0,5/13/11 3:32,"Renaissance art historian, University of Notti...",3115,NaN,9266CC,GabrieleNeher,NaN,https://pbs.twimg.com/profile_images/513327289...,0,FFFFFF,@Evielady just how lovely is the tree this yea...,NaN,26085,10/26/15 12:40,6.587300e+17,"Nottingham, England.",Amsterdam


In [4]:
twitter.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20050 entries, 0 to 20049
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   _unit_id               20050 non-null  int64  
 1   _golden                20050 non-null  bool   
 2   _unit_state            20050 non-null  object 
 3   _trusted_judgments     20050 non-null  int64  
 4   _last_judgment_at      20000 non-null  object 
 5   gender                 19953 non-null  object 
 6   gender:confidence      20024 non-null  float64
 7   profile_yn             20050 non-null  object 
 8   profile_yn:confidence  20050 non-null  float64
 9   created                20050 non-null  object 
 10  description            16306 non-null  object 
 11  fav_number             20050 non-null  int64  
 12  gender_gold            50 non-null     object 
 13  link_color             20050 non-null  object 
 14  name                   20050 non-null  object 
 15  pr

### Exploring data and cleaning it

In [5]:
# len(twitter['_golden'] > False)
twitter['_golden'].value_counts()

False    20000
True        50
Name: _golden, dtype: int64

In [6]:
twitter['_unit_state'].value_counts()

finalized    20000
golden          50
Name: _unit_state, dtype: int64

In [7]:
twitter['_trusted_judgments'].value_counts()

3      20000
245        4
249        4
264        4
259        4
260        3
242        3
247        2
227        2
235        2
261        1
274        1
258        1
212        1
273        1
228        1
256        1
271        1
262        1
246        1
255        1
231        1
248        1
233        1
250        1
252        1
237        1
253        1
254        1
270        1
239        1
240        1
Name: _trusted_judgments, dtype: int64

In [8]:
twitter['profile_yn'].value_counts()

yes    19953
no        97
Name: profile_yn, dtype: int64

In [9]:
twitter['gender_gold'].value_counts()

male               19
female             14
brand              13
unknown             2
female\nunknown     1
male\nunknown       1
Name: gender_gold, dtype: int64

In [10]:
twitter['profile_yn_gold'].value_counts()

yes    50
Name: profile_yn_gold, dtype: int64

In [11]:
twitter['tweet_coord'].value_counts()

[32.6192856, -96.8494873]      2
[25.2803, 51.5225]             2
[37.7463, -25.6668]            2
[44.4942, 11.3465]             2
[30.764, 20.2236]              2
                              ..
[25.94385047, -80.24606989]    1
[40.726541, -73.991416]        1
[52.42464889, 13.17969563]     1
[4.828516, 7.003183]           1
[42.3000412, -71.3850784]      1
Name: tweet_coord, Length: 146, dtype: int64

In [12]:
twitter['tweet_location'].value_counts()

London                152
United States         142
USA                    94
UK                     92
Los Angeles, CA        69
                     ... 
O'fallon, Illinois      1
North Raleigh, NC       1
Fresno, CA              1
eldoret                 1
Texas grapevine         1
Name: tweet_location, Length: 7864, dtype: int64

In [13]:
twitter['user_timezone'].value_counts()

Eastern Time (US & Canada)    2496
Pacific Time (US & Canada)    2106
Central Time (US & Canada)    1505
London                        1307
Atlantic Time (Canada)         589
                              ... 
Osaka                            1
Europe/Sarajevo                  1
Australia/Brisbane               1
PST                              1
GMT+3                            1
Name: user_timezone, Length: 156, dtype: int64

In [14]:
# twitter['user_timezone'].isna().sum()
twitter['user_timezone'].isnull().sum(axis = 0)

7798

In [15]:
twitter['description'].value_counts()

You can be spiritually empowered, financially secure, and improve your health all while doing so - our research proves it. Visit https://t.co/GrFOxFjwPV        33
Subscribe to her Inspirational channel here http://t.co/P1y6f2Rzoj ~~~~~~~~~~~~~~\nFor promoting on this page email, business_inquiry@outlook.com               32
secret little rendezvous                                                                                                                                        27
The Map Game is a free geography quiz based on Google Maps. No flash, no need to register, just start finding places on a blind map!                            24
'Cos even an old girl's best friend is still her MUM!                                                                                                           18
                                                                                                                                                                ..
I'm tired of the corru

In [16]:
# twitter['description'].isna().sum()
twitter['description'].isnull().sum(axis = 0)

3744

In [17]:
clean_df = twitter.drop(['_golden', '_unit_state', '_trusted_judgments', 'profile_yn', 'gender_gold', 'profile_yn_gold', 'tweet_coord', 'tweet_location', 'user_timezone'], axis = 1) 
# clean_df

In [18]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20050 entries, 0 to 20049
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   _unit_id               20050 non-null  int64  
 1   _last_judgment_at      20000 non-null  object 
 2   gender                 19953 non-null  object 
 3   gender:confidence      20024 non-null  float64
 4   profile_yn:confidence  20050 non-null  float64
 5   created                20050 non-null  object 
 6   description            16306 non-null  object 
 7   fav_number             20050 non-null  int64  
 8   link_color             20050 non-null  object 
 9   name                   20050 non-null  object 
 10  profileimage           20050 non-null  object 
 11  retweet_count          20050 non-null  int64  
 12  sidebar_color          20050 non-null  object 
 13  text                   20050 non-null  object 
 14  tweet_count            20050 non-null  int64  
 15  tw

In [19]:
clean_df.fillna(value = -1,  
          inplace = True) 

In [20]:
clean_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20050 entries, 0 to 20049
Data columns (total 17 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   _unit_id               20050 non-null  int64  
 1   _last_judgment_at      20050 non-null  object 
 2   gender                 20050 non-null  object 
 3   gender:confidence      20050 non-null  float64
 4   profile_yn:confidence  20050 non-null  float64
 5   created                20050 non-null  object 
 6   description            20050 non-null  object 
 7   fav_number             20050 non-null  int64  
 8   link_color             20050 non-null  object 
 9   name                   20050 non-null  object 
 10  profileimage           20050 non-null  object 
 11  retweet_count          20050 non-null  int64  
 12  sidebar_color          20050 non-null  object 
 13  text                   20050 non-null  object 
 14  tweet_count            20050 non-null  int64  
 15  tw

In [21]:
clean_df.head(20)

,_unit_id,_last_judgment_at,gender,gender:confidence,profile_yn:confidence,created,description,fav_number,link_color,name,profileimage,retweet_count,sidebar_color,text,tweet_count,tweet_created,tweet_id
0,815719226,10/26/15 23:24,male,1.0000,1.0,12/5/13 1:48,i sing my own rhythm.,0,08C2C2,sheezy0,https://pbs.twimg.com/profile_images/414342229...,0,FFFFFF,Robbie E Responds To Critics After Win Against...,110964,10/26/15 12:40,6.587300e+17
1,815719227,10/26/15 23:30,male,1.0000,1.0,10/1/12 13:51,I'm the author of novels filled with family dr...,68,0084B4,DavdBurnett,https://pbs.twimg.com/profile_images/539604221...,0,C0DEED,ÛÏIt felt like they were my friends and I was...,7471,10/26/15 12:40,6.587300e+17
2,815719228,10/26/15 23:33,male,0.6625,1.0,11/28/14 11:30,louis whining and squealing and all,7696,ABB8C2,lwtprettylaugh,https://pbs.twimg.com/profile_images/657330418...,1,C0DEED,i absolutely adore when louis starts the songs...,5617,10/26/15 12:40,6.587300e+17
3,815719229,10/26/15 23:10,male,1.0000,1.0,6/11/09 22:39,"Mobile guy. 49ers, Shazam, Google, Kleiner Pe...",202,0084B4,douggarland,https://pbs.twimg.com/profile_images/259703936...,0,C0DEED,Hi @JordanSpieth - Looking at the url - do you...,1693,10/26/15 12:40,6.587300e+17
4,815719230,10/27/15 1:15,female,1.0000,1.0,4/16/14 13:23,Ricky Wilson The Best FRONTMAN/Kaiser Chiefs T...,37318,3B94D9,WilfordGemma,https://pbs.twimg.com/profile_images/564094871...,0,0,Watching Neighbours on Sky+ catching up with t...,31462,10/26/15 12:40,6.587300e+17
5,815719231,10/27/15 1:47,female,1.0000,1.0,3/11/10 18:14,you don't know me.,3901,F5ABB5,monroevicious,https://pbs.twimg.com/profile_images/656336865...,0,0,"Ive seen people on the train with lamps, chair...",20036,10/26/15 12:40,6.587300e+17
6,815719232,10/27/15 1:57,brand,1.0000,1.0,4/24/08 13:03,"A global marketplace for images, videos and mu...",4122,298AAE,Shutterstock,https://pbs.twimg.com/profile_images/528547133...,0,0,@BpackEngineer Thank you for your patience whi...,13354,10/26/15 12:40,6.587300e+17
7,815719233,10/26/15 23:48,male,1.0000,1.0,12/3/12 21:54,The secret of getting ahead is getting started.,80,0000FF,RobinMeske,https://pbs.twimg.com/profile_images/508875440...,0,C0DEED,Gala Bingo clubs bought for å£241m: The UK's l...,112117,10/26/15 12:40,6.587300e+17
8,815719234,10/27/15 1:52,female,1.0000,1.0,9/8/15 4:50,Pll Fan // Crazy about MCD // Ramen is bae,1825,9266CC,pigzilla_,https://pbs.twimg.com/profile_images/658670112...,0,0,@_Aphmau_ the pic defines all mcd fangirls/fan...,482,10/26/15 12:40,6.587300e+17
9,815719235,10/27/15 1:49,female,1.0000,1.0,5/13/11 3:32,"Renaissance art historian, University of Notti...",3115,9266CC,GabrieleNeher,https://pbs.twimg.com/profile_images/513327289...,0,FFFFFF,@Evielady just how lovely is the tree this yea...,26085,10/26/15 12:40,6.587300e+17


In [22]:
# profile bio
twitter['description'][0]

'i sing my own rhythm.'

In [23]:
# profile random tweet
twitter['text'][1]

'\x89ÛÏIt felt like they were my friends and I was living the story with them\x89Û\x9d https://t.co/arngE0YHNO #retired #IAN1 https://t.co/CIzCANPQFz'

In [24]:
# clean_df = pd.DataFrame(data=twitter,columns=[])
# clean_df.drop(['', '', '', '', '', '', '', ''],1,inplace=True)

In [25]:
clean_df['gender'].value_counts()

female     6700
male       6194
brand      5942
unknown    1117
-1           97
Name: gender, dtype: int64

In [26]:
# clean_df['description'].isnull().sum()
# clean_df['text'].isnull().sum()

# both 0

# NLP Pipeline 

![](images/pipeline-walkthrough1.png)

Below is a to do list when converting text into vector form: 

**Clean text and Create a Bag of Words (BoW)**
>1. Lowercase the text
2. Tokenize 
3. Strip out punctuation or undesirable text
4. Remove Stopwords 
5. Stemming or Lemmatizing
6. Compute N-Grams
7. Use this to create BoW

**Vectorize BoW**
>8. Term Frequencies
9. Document Frequencies
10. TF-IDF
11. Normalize vectors

Let's go through both what each of these steps are and how to do them in python with the following corpus of comments about data science...
 

In [27]:
clean_df['text'].head(30)

0     Robbie E Responds To Critics After Win Against...
1     ÛÏIt felt like they were my friends and I was...
2     i absolutely adore when louis starts the songs...
3     Hi @JordanSpieth - Looking at the url - do you...
4     Watching Neighbours on Sky+ catching up with t...
5     Ive seen people on the train with lamps, chair...
6     @BpackEngineer Thank you for your patience whi...
7     Gala Bingo clubs bought for å£241m: The UK's l...
8     @_Aphmau_ the pic defines all mcd fangirls/fan...
9     @Evielady just how lovely is the tree this yea...
10    how are you taking care of yourself? https://t...
11    MTG Deals 1x Rank-Up-Magic - The Seventh One -...
12    Just put my ass on the line for you and this i...
13    https://t.co/z4sbWUugd8 What the Nation Will B...
14    will i even need sound effects for the diviner...
15    https://t.co/EROGWTFTYo It's a glow of satisfa...
16    @giannaaa28 lmao _Ù÷â_Ù÷â dude I'm hella scare...
17    @coolyazzy94 Ditto - I'm still learning th

### Create Courpus of Tweets AKA docs

In [28]:
clean_df['text']

0        Robbie E Responds To Critics After Win Against...
1        ÛÏIt felt like they were my friends and I was...
2        i absolutely adore when louis starts the songs...
3        Hi @JordanSpieth - Looking at the url - do you...
4        Watching Neighbours on Sky+ catching up with t...
                               ...                        
20045    @lookupondeath ...Fine, and I'll drink tea too...
20046    Greg Hardy you a good player and all but don't...
20047    You can miss people and still never want to se...
20048    @bitemyapp i had noticed your tendency to pee ...
20049    I think for my APUSH creative project I'm goin...
Name: text, Length: 20050, dtype: object

In [29]:
tweets = []
for i, doc in enumerate(clean_df['text']):
    tweets.append(clean_df['text'][i])
tweets

['Robbie E Responds To Critics After Win Against Eddie Edwards In The #WorldTitleSeries https://t.co/NSybBmVjKZ',
 '\x89ÛÏIt felt like they were my friends and I was living the story with them\x89Û\x9d https://t.co/arngE0YHNO #retired #IAN1 https://t.co/CIzCANPQFz',
 'i absolutely adore when louis starts the songs it hits me hard but it feels good',
 "Hi @JordanSpieth - Looking at the url - do you use @IFTTT?!  Don't typically see an advanced user on the @PGATOUR! https://t.co/H68ou5PE9L",
 'Watching Neighbours on Sky+ catching up with the Neighbs!! Xxx _Ù÷Ä_Ù÷Ä_Ù÷Ä_ÙÔÎ_Ù\x8fÈ_ÙÔ\x8d_Ù\x8fÈ Xxx',
 'Ive seen people on the train with lamps, chairs, tvs etc  https://t.co/w6zf4pVM4I',
 '@BpackEngineer Thank you for your patience while we take care of the issue!',
 "Gala Bingo clubs bought for å£241m: The UK's largest High Street bingo operator, Gala, is being taken over by\x89Û_ https://t.co/HzeeykJUd3",
 '@_Aphmau_ the pic defines all mcd fangirls/fanboys and mcd shippers xD',
 '@Evielady

In [30]:
len(tweets)

20050

### Emojis be Gone

In [31]:
# We will analyze not using emojis and re iterate through the same data with emojis

# let's start by removing emojis and other special characters
for i,doc in enumerate(tweets):
    tweets[i] = unicodedata.normalize('NFKD', doc).encode('ASCII', 'ignore').decode('utf8')

tweets

['Robbie E Responds To Critics After Win Against Eddie Edwards In The #WorldTitleSeries https://t.co/NSybBmVjKZ',
 'UIIt felt like they were my friends and I was living the story with themU https://t.co/arngE0YHNO #retired #IAN1 https://t.co/CIzCANPQFz',
 'i absolutely adore when louis starts the songs it hits me hard but it feels good',
 "Hi @JordanSpieth - Looking at the url - do you use @IFTTT?!  Don't typically see an advanced user on the @PGATOUR! https://t.co/H68ou5PE9L",
 'Watching Neighbours on Sky+ catching up with the Neighbs!! Xxx _UA_UA_UA_UOI_UE_UO_UE Xxx',
 'Ive seen people on the train with lamps, chairs, tvs etc  https://t.co/w6zf4pVM4I',
 '@BpackEngineer Thank you for your patience while we take care of the issue!',
 "Gala Bingo clubs bought for a241m: The UK's largest High Street bingo operator, Gala, is being taken over byU_ https://t.co/HzeeykJUd3",
 '@_Aphmau_ the pic defines all mcd fangirls/fanboys and mcd shippers xD',
 '@Evielady just how lovely is the tree thi

### Upper Case Be Gone

In [32]:
for i,doc in enumerate(tweets):
    tweets[i] = doc.lower()  

tweets

['robbie e responds to critics after win against eddie edwards in the #worldtitleseries https://t.co/nsybbmvjkz',
 'uiit felt like they were my friends and i was living the story with themu https://t.co/arnge0yhno #retired #ian1 https://t.co/cizcanpqfz',
 'i absolutely adore when louis starts the songs it hits me hard but it feels good',
 "hi @jordanspieth - looking at the url - do you use @ifttt?!  don't typically see an advanced user on the @pgatour! https://t.co/h68ou5pe9l",
 'watching neighbours on sky+ catching up with the neighbs!! xxx _ua_ua_ua_uoi_ue_uo_ue xxx',
 'ive seen people on the train with lamps, chairs, tvs etc  https://t.co/w6zf4pvm4i',
 '@bpackengineer thank you for your patience while we take care of the issue!',
 "gala bingo clubs bought for a241m: the uk's largest high street bingo operator, gala, is being taken over byu_ https://t.co/hzeeykjud3",
 '@_aphmau_ the pic defines all mcd fangirls/fanboys and mcd shippers xd',
 '@evielady just how lovely is the tree thi

### Be Gone @s and links

In [33]:
for i,doc in enumerate(tweets):
    tweets[i] = re.sub(r"(?:\@|https?\://)\S+", "", doc)
    tweets[i] = " ".join(tweets[i].split())
tweets

['robbie e responds to critics after win against eddie edwards in the #worldtitleseries',
 'uiit felt like they were my friends and i was living the story with themu #retired #ian1',
 'i absolutely adore when louis starts the songs it hits me hard but it feels good',
 "hi - looking at the url - do you use don't typically see an advanced user on the",
 'watching neighbours on sky+ catching up with the neighbs!! xxx _ua_ua_ua_uoi_ue_uo_ue xxx',
 'ive seen people on the train with lamps, chairs, tvs etc',
 'thank you for your patience while we take care of the issue!',
 "gala bingo clubs bought for a241m: the uk's largest high street bingo operator, gala, is being taken over byu_",
 'the pic defines all mcd fangirls/fanboys and mcd shippers xd',
 'just how lovely is the tree this year! never seen it as gorgeous as this #autumn #colour',
 'how are you taking care of yourself? #fitfluential',
 'mtg deals 1x rank-up-magic - the seventh one - prio-en058 - secret rare - unlimi',
 'just put my 

In [38]:
len(tweets)

20050

###  Hello to Tokenize

In [34]:
for i,doc in enumerate(tweets):
    tweets[i] = word_tokenize(doc) # method in nltk
tweets

[['robbie',
  'e',
  'responds',
  'to',
  'critics',
  'after',
  'win',
  'against',
  'eddie',
  'edwards',
  'in',
  'the',
  '#',
  'worldtitleseries'],
 ['uiit',
  'felt',
  'like',
  'they',
  'were',
  'my',
  'friends',
  'and',
  'i',
  'was',
  'living',
  'the',
  'story',
  'with',
  'themu',
  '#',
  'retired',
  '#',
  'ian1'],
 ['i',
  'absolutely',
  'adore',
  'when',
  'louis',
  'starts',
  'the',
  'songs',
  'it',
  'hits',
  'me',
  'hard',
  'but',
  'it',
  'feels',
  'good'],
 ['hi',
  '-',
  'looking',
  'at',
  'the',
  'url',
  '-',
  'do',
  'you',
  'use',
  'do',
  "n't",
  'typically',
  'see',
  'an',
  'advanced',
  'user',
  'on',
  'the'],
 ['watching',
  'neighbours',
  'on',
  'sky+',
  'catching',
  'up',
  'with',
  'the',
  'neighbs',
  '!',
  '!',
  'xxx',
  '_ua_ua_ua_uoi_ue_uo_ue',
  'xxx'],
 ['ive',
  'seen',
  'people',
  'on',
  'the',
  'train',
  'with',
  'lamps',
  ',',
  'chairs',
  ',',
  'tvs',
  'etc'],
 ['thank',
  'you',
  'for'

### Be Gone Punctuation

In [35]:
pt = string.punctuation
pt

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [36]:
for i,doc in enumerate(tweets):
    tweets[i] = [word for word in doc if not word in pt]
tweets

[['robbie',
  'e',
  'responds',
  'to',
  'critics',
  'after',
  'win',
  'against',
  'eddie',
  'edwards',
  'in',
  'the',
  'worldtitleseries'],
 ['uiit',
  'felt',
  'like',
  'they',
  'were',
  'my',
  'friends',
  'and',
  'i',
  'was',
  'living',
  'the',
  'story',
  'with',
  'themu',
  'retired',
  'ian1'],
 ['i',
  'absolutely',
  'adore',
  'when',
  'louis',
  'starts',
  'the',
  'songs',
  'it',
  'hits',
  'me',
  'hard',
  'but',
  'it',
  'feels',
  'good'],
 ['hi',
  'looking',
  'at',
  'the',
  'url',
  'do',
  'you',
  'use',
  'do',
  "n't",
  'typically',
  'see',
  'an',
  'advanced',
  'user',
  'on',
  'the'],
 ['watching',
  'neighbours',
  'on',
  'sky+',
  'catching',
  'up',
  'with',
  'the',
  'neighbs',
  'xxx',
  '_ua_ua_ua_uoi_ue_uo_ue',
  'xxx'],
 ['ive',
  'seen',
  'people',
  'on',
  'the',
  'train',
  'with',
  'lamps',
  'chairs',
  'tvs',
  'etc'],
 ['thank',
  'you',
  'for',
  'your',
  'patience',
  'while',
  'we',
  'take',
  'care'

In [43]:
twitter['text'][3], tweets[3]

("Hi @JordanSpieth - Looking at the url - do you use @IFTTT?!  Don't typically see an advanced user on the @PGATOUR! https://t.co/H68ou5PE9L",
 ['hi',
  'looking',
  'at',
  'the',
  'url',
  'do',
  'you',
  'use',
  'do',
  "n't",
  'typically',
  'see',
  'an',
  'advanced',
  'user',
  'on',
  'the'])

### Be Gone Empty Spaces

In [45]:
for i,doc in enumerate(tweets):
    if '' in doc:
        tweets.remove('') 
# tweets

### Remove Stopwords - Be Gone Stopwords

Sometimes, some extremely common words which would appear to be of little value in understanding the content of language are excluded from the vocabulary entirely. These words are called **stop words**.

**Examples:**
1. the
2. is
3. to

In [47]:
sw = stopwords.words('english') # stopwords from nltk
sw;

In [49]:
for i,doc in enumerate(tweets):
    tweets[i] = [token for token in doc if token not in sw]
# tweets